# Traffic Light Detection and Classification
Using a pre-trained model to detect objects in an image.

In [ ]:
import numpy as np
import os
import sys
import tensorflow as tf
import time

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
%matplotlib inline

## Model preparation

In [ ]:
ssd_inception_udacity_model = 'udacity_data_model/frozen_inference_graph.pb'

PATH_TO_LABELS = 'label_map_red_light_only.pbtxt'

NUM_CLASSES = 1

## Detection

## 1. Testing SSD Inception Models

### 1.1 Testing model trained on real images on Real World images

In [ ]:
detection_graph = tf.Graph()

with detection_graph.as_default():
    
  od_graph_def = tf.GraphDef()

  with tf.gfile.GFile(ssd_inception_udacity_model, 'rb') as fid:
        
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [ ]:
import cv2
def process_image(image):
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            # Definite input and output Tensors for detection_graph
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

            # Each box represents a part of the image where a particular object was detected.
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image, axis=0)

            time0 = time.time()

            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
            
            
            height = image.shape[0]
            width = image.shape[1]
            
            boxes = np.squeeze(boxes)
            scores = np.squeeze(scores)
            classes = np.squeeze(classes).astype(np.int32)

            min_score_thresh = .50
            for i in range(boxes.shape[0]):
                if scores is None or scores[i] > min_score_thresh:
                    x_min = int(boxes[i][1]*width)
                    y_min = int(boxes[i][0]*height)
                    x_max = int(boxes[i][3]*width)
                    y_max = int(boxes[i][2]*height)
                    cv2.rectangle(image,(x_min,y_min),(x_max,y_max),(0,255,0),1)
    return image
            

In [ ]:
from moviepy.editor import VideoFileClip
white_output = 'output_video.mp4'

clip1 = VideoFileClip("test_lot.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)